In [1]:
from theano.sandbox import cuda

%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: CUDA driver version is insufficient for CUDA runtime version)
Using TensorFlow backend.


In [23]:
notre_dame = "http://www.gutenberg.org/cache/epub/19657/pg19657.txt"
lesmiserable1 = "http://www.gutenberg.org/cache/epub/17489/pg17489.txt"

notredame_path = get_file('nd.txt', origin=str(notre_dame))
import codecs

text = codecs.open(notredame_path, "r", "utf-8").read()
print('corpus length:', len(text))
print(text[1:100])

corpus length: 1087507
The Project Gutenberg EBook of Notre-Dame de Paris, by Victor Hugo

This eBook is for the use of 


In [40]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print('total chars:', vocab_size)
for i in chars[-6:]: print(i)

total chars: 115
ï
ñ
ô
ù
û
﻿


In [41]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [45]:
idx = [char_indices[c] for i,c in enumerate(text)]
idx[1:10]

[49, 66, 63, 2, 45, 76, 73, 68, 63]

In [46]:
"".join(indices_char[i] for i in idx[1:50])

u'The Project Gutenberg EBook of Notre-Dame de Pari'

# Simple RNN 

In [47]:
embedding_size = 42
hidden_layers = 256
character_set = 8
vocab_size = len(chars)+1

In [49]:
model=Sequential([
        Embedding(vocab_size, embedding_size, input_length=character_set),
        SimpleRNN(hidden_layers, activation='relu', inner_init='identity'),
        Dense(vocab_size, activation='softmax')
    ])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 8, 42)         4830        embedding_input_2[0][0]          
____________________________________________________________________________________________________
simplernn_2 (SimpleRNN)          (None, 256)           76544       embedding_2[0][0]                
____________________________________________________________________________________________________
dense_2 (Dense)                  (None, 115)           29555       simplernn_2[0][0]                
Total params: 110,929
Trainable params: 110,929
Non-trainable params: 0
____________________________________________________________________________________________________


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())


## Preprocess input and output

In [146]:
type(idx[0])

int

In [150]:
#  from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py
char_in = []
char_out = []
for i in range(0, len(text) - 1 - character_set, character_set):
    char_in.append([char_indices[c] for c in text[i: i + character_set]])
    char_out.append(char_indices[text[i + character_set]])
print('nb sequences:', len(sequences))

sequences[1]
# x= np.zeros(character_set,)
# y =[]
# for i, seq in enumerate(sequences):
#     for j, char in enumerate(seq):
#         x[]


nb sequences: 135938


[68, 63, 61, 78, 2, 36, 79, 78]

## Fit

In [151]:
model.optimizer.lr=0.001
model.fit(char_in, char_out, batch_size=64, nb_epoch=8)

Epoch 1/8
105408/135938 [======================>.......] - ETA: 6s - loss: 2.6985

KeyboardInterrupt: 